In [ ]:
# !pip install tensorflow==2.15.0

In [1]:
import matplotlib.pylab as pylab
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import tqdm
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [2]:
print(tf.__version__)

2.15.0


In [4]:
## tensorflow 2 ###

# Grid search
# import talos

df = pd.read_csv('/content/drive/MyDrive/capstone/data/decompose_by_interest_4.csv')

# df = pd.read_csv('/content/drive/My Drive/decompose_by_attention_per_day.csv')
# df_at = pd.read_csv('/content/drive/My Drive/decompose_by_attention.csv')
# df_ca = pd.read_csv('/content/drive/My Drive/decompose_by_caution.csv')
# df_se = pd.read_csv('/content/drive/My Drive/decompose_by_serious.csv')

numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

print(df.shape)
df

(1471716, 38)


,datetime,fw,wl,wlobscd,ymdhm,observatory,interest,attention,caution,serious,...,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour,rf_kangwon_4hour,rf_kangwon_6hour,rf_kangwon_8hour,rf_kangwon_10hour,rf_kangwon_12hour
0,2010-01-01 00:00:00,6.08,1.78,1018655,2010010100,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
1,2010-01-01 01:00:00,6.08,1.78,1018655,2010010101,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
2,2010-01-01 02:00:00,6.08,1.78,1018655,2010010102,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
3,2010-01-01 03:00:00,6.08,1.78,1018655,2010010103,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
4,2010-01-01 04:00:00,6.08,1.78,1018655,2010010104,Daegok Bridge,3.8,5.5,7.0,8.5,...,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471711,2023-12-31 19:00:00,128.90,1.13,1018640,2023123119,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,0.200,11.507,130.316,129.862,0.200,0.0,0.0,0.0,0.8,0.0
1471712,2023-12-31 20:00:00,124.87,1.13,1018640,2023123120,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,127.950,0.200,118.564,139.239,1.120,0.0,0.0,2.7,0.0,0.0
1471713,2023-12-31 21:00:00,124.87,1.13,1018640,2023123121,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,142.493,0.200,11.507,130.316,129.862,0.0,0.0,0.0,0.0,0.8
1471714,2023-12-31 22:00:00,128.90,1.13,1018640,2023123122,Gwangjin Bridge,3.0,8.0,10.5,13.6,...,143.902,127.950,0.200,118.564,139.239,0.0,0.0,0.0,2.7,0.0


In [5]:
warning_level = df[['observatory','interest','attention','caution','serious']]
warning_level = warning_level.drop_duplicates().reset_index(drop=True)
df.drop(['interest','attention','caution','serious'], axis=1, inplace=True)
warning_level

,observatory,interest,attention,caution,serious
0,Daegok Bridge,3.8,5.5,7.0,8.5
1,Ogeum Bridge,4.0,7.0,9.0,11.1
2,Neobu Bridge,1.5,3.0,4.0,4.8
3,Haengju Bridge,4.5,6.5,8.0,10.7
4,Daechi Bridge,3.0,8.5,11.0,13.5
5,Jamsu Bridge,3.9,5.5,6.2,6.5
6,Hangang Bridge,3.9,8.5,10.5,13.3
7,Wolgye 2 Bridge,2.0,5.0,6.0,6.2
8,Changdong Bridge,4.5,5.5,6.5,7.9
9,Cheongdam Bridge,3.0,9.0,11.5,16.0


In [22]:
from sklearn.preprocessing import StandardScaler

# StandardScaler를 사용하여 데이터 정규화

col = ['wl', 'fw', 'rf_seoul','paldang_4hour', 'chungju_4hour', 'rf_kangwon_4hour','residual','seasonal','trend']

scaler = StandardScaler()
scaler.fit(df[col])

# 원래 데이터의 평균과 표준편차 저장
mean_values = scaler.mean_
std_values = scaler.scale_

# 데이터 정규화
df[col] = scaler.transform(df[col])
df

,datetime,fw,wl,wlobscd,ymdhm,observatory,latitude,longitude,date,year,...,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour,rf_kangwon_4hour,rf_kangwon_6hour,rf_kangwon_8hour,rf_kangwon_10hour,rf_kangwon_12hour
0,2010-01-01 00:00:00,-0.378675,0.557043,1018655,2010010100,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1,2010-01-01 01:00:00,-0.378675,0.557043,1018655,2010010101,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
2,2010-01-01 02:00:00,-0.378675,0.557043,1018655,2010010102,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
3,2010-01-01 03:00:00,-0.378675,0.557043,1018655,2010010103,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
4,2010-01-01 04:00:00,-0.378675,0.557043,1018655,2010010104,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410390,2016-12-31 19:00:00,0.838913,-0.343824,1018640,2016123119,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410391,2016-12-31 20:00:00,0.838913,-0.343824,1018640,2016123120,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410392,2016-12-31 21:00:00,0.838913,-0.343824,1018640,2016123121,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410393,2016-12-31 22:00:00,0.838913,-0.343824,1018640,2016123122,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0


In [23]:
# 변환된 값을 다시 원래 값으로 되돌리기 위한 함수 정의
def inverse_transform_standardized(scaled_values, mean_values, std_values):
    original_values = scaled_values * std_values + mean_values
    return original_values

In [24]:
print(mean_values, std_values)

[ 1.29151080e+00  3.11392161e+02  1.58549949e-01  3.48529811e+00
  4.38933808e+02  1.53977365e-01 -6.80512147e-05 -2.84430434e-06
  1.28849457e+00] [8.76933300e-01 8.06265180e+02 1.37666043e+00 3.07783372e+01
 9.32412870e+02 1.24622868e+00 3.31027520e-01 1.91443248e-01
 7.63086421e-01]


In [25]:

lossmetric = 'mean_absolute_error'
sequence_length = 8
dropae = 0.1

In [26]:
def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]


def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

In [27]:
### CREATE TRAIN/TEST DATA ###
X, y = [], []
for sequence in gen_sequence(df[df["observatory"] == 'Cheongdam Bridge'], sequence_length, ['wl']):
    X.append(sequence)
X = np.asarray(X)

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

for sequence in gen_labels(df[df["observatory"] == 'Cheongdam Bridge'], sequence_length, ['wl']):
    y.append(sequence)
y = np.asarray(y)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

# X = df.drop(['wl'], axis=1)
# y = df['wl']
# X_train , X_test = train_test_split(X, train_size=0.8, random_state=42)
# y_train , y_test = train_test_split(y, train_size=0.8, random_state=42)

In [28]:
# X_train = np.asarray(X_train)
# X_test = np.asarray(X_test)
# y_train = []
# y_test = []
# y_other_train = []
# y_other_test = []

# for bridge in df["observatory"].unique():
#     for sequence in gen_sequence(df[df["observatory"] == bridge], sequence_length, ['wl']):
#         if bridge in trainBridge:
#             y_train.append(sequence)
#     for sequence in gen_sequence(df[df["observatory"] == bridge], sequence_length, ['wl']):
#         if bridge in testBridge:
#             y_test.append(sequence)

In [29]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [30]:
from datetime import datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
df

,datetime,fw,wl,wlobscd,ymdhm,observatory,latitude,longitude,date,year,...,paldang_4hour,paldang_6hour,paldang_8hour,paldang_10hour,paldang_12hour,rf_kangwon_4hour,rf_kangwon_6hour,rf_kangwon_8hour,rf_kangwon_10hour,rf_kangwon_12hour
0,2010-01-01 00:00:00,-0.378675,0.557043,1018655,2010010100,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1,2010-01-01 01:00:00,-0.378675,0.557043,1018655,2010010101,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
2,2010-01-01 02:00:00,-0.378675,0.557043,1018655,2010010102,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
3,2010-01-01 03:00:00,-0.378675,0.557043,1018655,2010010103,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
4,2010-01-01 04:00:00,-0.378675,0.557043,1018655,2010010104,Daegok Bridge,37.468742,127.122572,2010-01-01,2010,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410390,2016-12-31 19:00:00,0.838913,-0.343824,1018640,2016123119,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410391,2016-12-31 20:00:00,0.838913,-0.343824,1018640,2016123120,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410392,2016-12-31 21:00:00,0.838913,-0.343824,1018640,2016123121,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0
1410393,2016-12-31 22:00:00,0.838913,-0.343824,1018640,2016123122,Gwangjin Bridge,37.542950,127.113154,2016-12-31,2016,...,-0.113239,0.0,0.0,0.0,0.0,-0.123555,0.0,0.0,0.0,0.0


In [31]:
df['year'].unique()
splitTest = 5105
customXLabel = []
for j in range(len(df['year'].unique())-1):
    j = j*12
    customXLabel.append(j)
customXLabel

[0, 12, 24, 36, 48, 60]

In [32]:
### CONCATENATE TRAIN/TEST DATA AND LABEL ###
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

In [33]:
F = []

for sequence in gen_sequence(df[df['observatory']=='Cheongdam Bridge'], sequence_length, col):
    F.append(sequence)
F = np.asarray(F)
f_train, f_test = train_test_split(F, test_size=0.2, random_state=42)

In [34]:
f_train = np.asarray(f_train)
f_test = np.asarray(f_test)

F = np.concatenate([f_train,f_test], axis=0)

In [35]:
print(X.shape, X_train.shape, X_test.shape)
print(y.shape, y_train.shape, y_test.shape)
print(F.shape, f_train.shape, f_test.shape)

(61314, 8, 1) (49051, 8, 1) (12263, 8, 1)
(61314, 1) (49051, 1) (12263, 1)
(61314, 8, 9) (49051, 8, 9) (12263, 8, 9)


In [36]:
### SET SEED ###
os.environ['PYTHONHASHSEED'] = str(47)
np.random.seed(47)
random.seed(47)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(),
    config=session_conf
)
tf.compat.v1.keras.backend.set_session(sess)

In [37]:
### LSTM AUTOENCODER ###
inputsAE = Input(shape=(sequence_length, 1))

encodedAE = GRU(128, return_sequences=True, dropout=dropae)(inputsAE, training=True)

decodedAE = GRU(32, return_sequences=True, dropout=dropae)(encodedAE, training=True)

outputAE = TimeDistributed(Dense(1))(decodedAE)

sequenceAE = Model(inputsAE, outputAE)
sequenceAE.compile(optimizer='adam', loss=lossmetric, metrics=[lossmetric])

In [38]:
### TRAIN AUTOENCODER ###
sequenceAE.fit(X[:len(X_train)], X[:len(X_train)], batch_size=128, epochs=30, verbose=2, shuffle=False)

os.environ['PYTHONHASHSEED'] = str(47)
np.random.seed(47)
random.seed(47)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(),
    config=session_conf
)
tf.compat.v1.keras.backend.set_session(sess)

Epoch 1/30
384/384 - 24s - loss: 0.0676 - mean_absolute_error: 0.0676 - 24s/epoch - 63ms/step
Epoch 2/30
384/384 - 11s - loss: 0.0425 - mean_absolute_error: 0.0425 - 11s/epoch - 29ms/step
Epoch 3/30
384/384 - 10s - loss: 0.0394 - mean_absolute_error: 0.0394 - 10s/epoch - 27ms/step
Epoch 4/30
384/384 - 9s - loss: 0.0368 - mean_absolute_error: 0.0368 - 9s/epoch - 24ms/step
Epoch 5/30
384/384 - 11s - loss: 0.0354 - mean_absolute_error: 0.0354 - 11s/epoch - 28ms/step
Epoch 6/30
384/384 - 10s - loss: 0.0341 - mean_absolute_error: 0.0341 - 10s/epoch - 25ms/step
Epoch 7/30
384/384 - 10s - loss: 0.0340 - mean_absolute_error: 0.0340 - 10s/epoch - 25ms/step
Epoch 8/30
384/384 - 11s - loss: 0.0334 - mean_absolute_error: 0.0334 - 11s/epoch - 28ms/step
Epoch 9/30
384/384 - 9s - loss: 0.0330 - mean_absolute_error: 0.0330 - 9s/epoch - 24ms/step
Epoch 10/30
384/384 - 10s - loss: 0.0323 - mean_absolute_error: 0.0323 - 10s/epoch - 27ms/step
Epoch 11/30
384/384 - 14s - loss: 0.0320 - mean_absolute_error:

In [39]:
"""# GRU after AutoEncoder Urban & Rural"""

os.environ['PYTHONHASHSEED'] = str(47)
np.random.seed(47)
random.seed(47)

dropOut = 0.5
epochs = 200
lstmcell1 = 256
lstmcell2 = 128
dense1 = 64

In [40]:
encoder = Model(inputsAE, encodedAE)
XX = encoder.predict(X)
XXF = np.concatenate([XX, F], axis=2)
XXF.shape

1917/1917 [==============================] - 11s 6ms/step


(61314, 8, 137)

In [41]:
X_train1, X_test1 = XXF[:len(X_train)], XXF[len(X_train):]
y_train1, y_test1 = y[:len(y_train)], y[len(y_train):]

scaler1 = StandardScaler()

X_train1 = scaler1.fit_transform(X_train1.reshape(-1, XXF.shape[-1])).reshape(-1, sequence_length, XXF.shape[-1])
X_test1 = scaler1.transform(X_test1.reshape(-1, XXF.shape[-1])).reshape(-1, sequence_length, XXF.shape[-1])

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(),
    config=session_conf
)
tf.compat.v1.keras.backend.set_session(sess)

In [42]:
### DEFINE FORECASTER ###
inputs1 = Input(shape=(X_train1.shape[1], X_train1.shape[2]))
lstm1 = GRU(lstmcell1, return_sequences=True,
            dropout=dropOut)(inputs1, training=True)
lstm1 = GRU(lstmcell2, return_sequences=False,
            dropout=dropOut)(lstm1, training=True)
dense1 = Dense(dense1)(lstm1)
out1 = Dense(1)(dense1)

model1 = Model(inputs1, out1)
model1.compile(loss=lossmetric, optimizer='adam', metrics=[lossmetric, 'accuracy'])

In [ ]:
### FIT FORECASTER ###
train_size = int(len(X_train1) * 0.8)

history = model1.fit(X_train1[:train_size], y_train1[:train_size],
                     validation_data=(X_train1[train_size:], y_train1[train_size:]),
                     epochs=epochs, batch_size=256, verbose=2, shuffle=False)

Epoch 1/200
154/154 - 34s - loss: 0.1692 - mean_absolute_error: 0.1692 - accuracy: 0.0000e+00 - val_loss: 0.1295 - val_mean_absolute_error: 0.1295 - val_accuracy: 0.0000e+00 - 34s/epoch - 221ms/step
Epoch 2/200
154/154 - 28s - loss: 0.1210 - mean_absolute_error: 0.1210 - accuracy: 0.0000e+00 - val_loss: 0.1212 - val_mean_absolute_error: 0.1212 - val_accuracy: 0.0000e+00 - 28s/epoch - 179ms/step
Epoch 3/200
154/154 - 26s - loss: 0.1149 - mean_absolute_error: 0.1149 - accuracy: 0.0000e+00 - val_loss: 0.1182 - val_mean_absolute_error: 0.1182 - val_accuracy: 0.0000e+00 - 26s/epoch - 170ms/step
Epoch 4/200
154/154 - 27s - loss: 0.1089 - mean_absolute_error: 0.1089 - accuracy: 0.0000e+00 - val_loss: 0.1101 - val_mean_absolute_error: 0.1101 - val_accuracy: 0.0000e+00 - 27s/epoch - 178ms/step
Epoch 5/200
154/154 - 27s - loss: 0.1069 - mean_absolute_error: 0.1069 - accuracy: 0.0000e+00 - val_loss: 0.1078 - val_mean_absolute_error: 0.1078 - val_accuracy: 0.0000e+00 - 27s/epoch - 178ms/step
Epoch

In [ ]:
plt.plot(history.history['loss'][10:])
plt.plot(history.history['val_loss'][10:])

plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper left')
plt.show()

In [ ]:
def test_other_drop1(X_test, y_test, f_test, enc, NN):
    X = X_test
    F = f_test
    y = y_test
    enc_pred = np.vstack(enc([X]))
    enc_pred = np.concatenate([enc_pred, F], axis=2)

    trans_pred = scaler1.transform(enc_pred.reshape(-1, enc_pred.shape[-1])).reshape(-1, sequence_length, enc_pred.shape[-1])
    NN_pred = NN([trans_pred])

    return np.vstack(NN_pred), y

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    common_indices = np.intersect1d(y_true.index, y_pred.index)
    y_true = y_true.loc[common_indices]
    y_pred = y_pred.loc[common_indices]
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 8),
          'axes.labelsize': '15',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': '18',
          'ytick.labelsize': '22'}
pylab.rcParams.update(params)

In [ ]:
data = df

In [ ]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error, r2_score
import gc

gc.collect()

submission_list = []
inversed_submission_list = []

bridge_name = 'Cheongdam Bridge'
# b = df[(df["observatory"] == j)]

X_test = []
y_test = []
f_test = []
sequence_length = 8
df = data[data['year'] <= 2016]

### X_test ###
for sequence in gen_sequence(df[df['observatory']=='Cheongdam Bridge'],sequence_length, ['wl']):
    X_test.append(sequence)
X_test = np.asarray(X_test)
print(X_test.shape)

for sequence in gen_labels(df[df['observatory']=='Cheongdam Bridge'],sequence_length, ['wl']):
    y_test.append(sequence)
y_test = np.asarray(y_test)
print(y_test.shape)

### f_test ###
for sequence in gen_sequence(df[df['observatory']=='Cheongdam Bridge'],sequence_length, col):
    f_test.append(sequence)
f_test = np.asarray(f_test)
print(f_test.shape)

print(X_test.shape, y_test.shape, f_test.shape)

R2, MAE, MSE, RMSE = [], [], [] ,[]
trueValuesList = []

enc = K.function([encoder.layers[0].input], [encoder.layers[-1].output])
NN = K.function([model1.layers[0].input], [model1.layers[-1].output])

for i in tqdm.tqdm(range(0, 100)):
    predictedValues, trueValues = test_other_drop1(X_test, y_test, f_test, enc, NN)
    R2.append(r2_score(predictedValues, trueValues))
    MAE.append(mean_absolute_error(predictedValues, trueValues))
    MSE.append(mean_squared_error(predictedValues, trueValues))
    RMSE.append(np.sqrt(mean_squared_error(predictedValues, trueValues)))
    trueValuesList.append(trueValues)

print("\tR-squared : %.4f (Mean) %.4f (Standard Error)" % (np.mean(R2), np.std(R2)))
print("\tMAE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(MAE), np.std(MAE)))
print("\tMSE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(MSE), np.std(MSE)))
print("\tRMSE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(RMSE), np.std(RMSE)))

#### 그래프 그리기 ####
plt.rcParams['figure.figsize'] = 20, 8
f, ax = plt.subplots()

submission = pd.DataFrame()
submission['true'] = trueValues.flatten()
submission['predict'] = predictedValues.flatten()
submission['bridge_name'] = [bridge_name for _ in range(len(trueValues))]
submission_list.append(submission)

inverse_true = inverse_transform_standardized(np.mean(np.hstack(trueValuesList).T, axis=0), mean_values[0], std_values[0])
ax.plot(inverse_true, color='black', label='Water Level', marker='x')

inverse_predict = inverse_transform_standardized(predictedValues, mean_values[0], std_values[0])
ax.plot(inverse_predict, color='red', label="Prediction", marker='d')

ax.set_title(bridge_name)

w = warning_level[warning_level['observatory'] == bridge_name]

inversed_submission = pd.DataFrame()
inversed_submission['true'] = inverse_true.flatten()
inversed_submission['predict'] = inverse_predict.flatten()
inversed_submission['bridge_name'] = [bridge_name for _ in range(len(inverse_true))]
inversed_submission['anomaly'] = inversed_submission['true'] > w['interest'].iloc[0]
inversed_submission_list.append(inversed_submission)

ax.axhline(y=w['interest'].item(), color='green', linestyle='--', label='interest')
ax.axhline(y=w['attention'].item(), color='yellow', linestyle='--', label='attention')
ax.axhline(y=w['caution'].item(), color='orange', linestyle='--', label='caution')
ax.axhline(y=w['serious'].item(), color='red', linestyle='--', label='serious')

ax.legend()

plt.grid(color='k', linestyle='-', linewidth=0.4)
plt.tight_layout()
plt.savefig('[interest]' + bridge_name +'_DropOut_'+str(dropOut) + 'only daegyo '+'.png')
plt.show()
x = 0


In [ ]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error, r2_score
import gc

gc.collect()

bridge_name = 'Cheongdam Bridge'
# b = df[(df["observatory"] == j)]

X_test = []
y_test = []
f_test = []
sequence_length = 8
df = data[data['year'] > 2016]
### X_test ###
for sequence in gen_sequence(df[df['observatory']=='Cheongdam Bridge'],sequence_length, ['wl']):
    X_test.append(sequence)
X_test = np.asarray(X_test)
print(X_test.shape)

for sequence in gen_labels(df[df['observatory']=='Cheongdam Bridge'],sequence_length, ['wl']):
    y_test.append(sequence)
y_test = np.asarray(y_test)
print(y_test.shape)

### f_test ###
for sequence in gen_sequence(df[df['observatory']=='Cheongdam Bridge'],sequence_length, col):
    f_test.append(sequence)
f_test = np.asarray(f_test)
print(f_test.shape)

print(X_test.shape, y_test.shape, f_test.shape)

R2, MAE, MSE, RMSE = [], [], [] ,[]
trueValuesList = []

enc = K.function([encoder.layers[0].input], [encoder.layers[-1].output])
NN = K.function([model1.layers[0].input], [model1.layers[-1].output])

for i in tqdm.tqdm(range(0, 100)):
    predictedValues, trueValues = test_other_drop1(X_test, y_test, f_test, enc, NN)
    R2.append(r2_score(predictedValues, trueValues))
    MAE.append(mean_absolute_error(predictedValues, trueValues))
    MSE.append(mean_squared_error(predictedValues, trueValues))
    RMSE.append(np.sqrt(mean_squared_error(predictedValues, trueValues)))
    trueValuesList.append(trueValues)

print("\tR-squared : %.4f (Mean) %.4f (Standard Error)" % (np.mean(R2), np.std(R2)))
print("\tMAE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(MAE), np.std(MAE)))
print("\tMSE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(MSE), np.std(MSE)))
print("\tRMSE : %.4f (Mean) %.4f (Standard Error)" % (np.mean(RMSE), np.std(RMSE)))

#### 그래프 그리기 ####
plt.rcParams['figure.figsize'] = 20, 8
f, ax = plt.subplots()

submission = pd.DataFrame()
submission['true'] = trueValues.flatten()
submission['predict'] = predictedValues.flatten()
submission['bridge_name'] = [bridge_name for _ in range(len(trueValues))]
submission_list.append(submission)

inverse_true = inverse_transform_standardized(np.mean(np.hstack(trueValuesList).T, axis=0), mean_values[0], std_values[0])
ax.plot(inverse_true, color='black', label='Water Level', marker='x')

inverse_predict = inverse_transform_standardized(predictedValues, mean_values[0], std_values[0])
ax.plot(inverse_predict, color='red', label="Prediction", marker='d')

ax.set_title(bridge_name)

w = warning_level[warning_level['observatory'] == bridge_name]

inversed_submission = pd.DataFrame()
inversed_submission['true'] = inverse_true.flatten()
inversed_submission['predict'] = inverse_predict.flatten()
inversed_submission['bridge_name'] = [bridge_name for _ in range(len(inverse_true))]
inversed_submission['anomaly'] = inversed_submission['true'] > w['interest'].iloc[0]
inversed_submission_list.append(inversed_submission)

ax.axhline(y=w['interest'].item(), color='green', linestyle='--', label='interest')
ax.axhline(y=w['attention'].item(), color='yellow', linestyle='--', label='attention')
ax.axhline(y=w['caution'].item(), color='orange', linestyle='--', label='caution')
ax.axhline(y=w['serious'].item(), color='red', linestyle='--', label='serious')

ax.legend()

plt.grid(color='k', linestyle='-', linewidth=0.4)
plt.tight_layout()
plt.savefig('[interest]' + bridge_name +'_DropOut_'+str(dropOut) + 'only daegyo '+'.png')
plt.show()
x = 0


In [ ]:
import pandas as pd
submission = pd.concat(submission_list)
inversed_submission = pd.concat(inversed_submission_list)

submission['anomaly'] = inversed_submission['anomaly']

submission.to_csv('/content/drive/MyDrive/capstone/data/scaled_submission_Cheongdam_10years_3(interest).csv', index=False, encoding='utf-8')
inversed_submission.to_csv('/content/drive/MyDrive/capstone/data/inversed_submission_Cheongdam_10years_3(interest).csv', index=False, encoding='utf-8')